In [3]:
dataPath='/home/khrovatin/timeTrajectoriesNet/data/counts/'
dataPathNormalised='/home/khrovatin/timeTrajectoriesNet/data/RPKUM/'
dataPathCode='/home/khrovatin/git/baylor-dicty/R/'

In [4]:
library("DESeq2")
library('dplyr')
source(paste(dataPathCode,'libraryDE.R',sep=''))

library("BiocParallel")
register(MulticoreParam(4))


In [5]:
#Load data counts
genes<-read.table(paste(dataPath,"mergedGenes_counts.tsv",sep=''), header=TRUE,row.names=1, sep="\t")
conditions<-read.table(paste(dataPathNormalised,"conditions_mergedGenes.tsv",sep=''), header=TRUE,row.names='Measurment', sep="\t")
#R imported colnames of genes with changes but gene IDs remained ok
rownames(conditions)<-make.names(rownames(conditions))
#Load data normalised
genesNorm<-read.table(paste(dataPathNormalised,"mergedGenes_RPKUM.tsv",sep=''), header=TRUE,row.names=1, sep="\t")

STAGES<-c('no_agg','stream','lag','tag','tip','slug','mhat','cul','FB','yem')
PHENOTYPES_ORDERED=c('no_agg', 'stream', 'lag', 'tag', 'tip', 'slug', 'mhat', 'cul', 'FB','yem')
PHENOTYPES_X=data.frame(Phenotype=PHENOTYPES_ORDERED,X=c(1:length(PHENOTYPES_ORDERED)))

In [12]:
#*** Find stage specific genes 1 vs all adjusting for replicates (Do in WT or all)
#Uses only replicates present in both stage and rest group
# For stage A use timepoints with phenotypes A and A/B for test and everything without A as control (B, B/C)

#WT or all - also change the path for saving below
conditions_test=conditions
#conditions_test=conditions[conditions$Group=='WT',]
conditions_test=conditions_test[rowSums(conditions_test[, PHENOTYPES_ORDERED])>0, ]
for (stage in STAGES){
  print(stage)
  if (sum(conditions_test[stage])>0){
    test<-conditions_test[conditions_test[stage]==1,]
    #Select only measurments where 'stage' is the only phenotype e.g. test only A (not A/B)
    #single_stage1<-rowSums(test[,STAGES])==1
    #test<-test[single_stage1,]
    control<-conditions_test[conditions_test[stage]!=1,]
    replicates<-intersect(control$Replicate,test$Replicate)
    print(paste('N rep',length(unique(replicates))))
    test<-test[test$Replicate %in% replicates,]
    control<-control[control$Replicate %in% replicates,]
    test$Comparison<-rep(stage,dim(test)[1])
    control$Comparison<-rep('other',dim(control)[1])
    conditions_sub<-rbind(test,control)
    genes_sub<-genes[,rownames(conditions_sub)]
    print(dim(conditions_sub))
    res<-runDeSeq2(conditions_sub,genes_sub,case=stage,control='other',design=~Replicate+Comparison,main_lvl='Comparison',padj=0.05,logFC=1,
                   path='/home/khrovatin/timeTrajectoriesNet/data/deTime/stage_vs_other/')
  } 
} 


[1] "no_agg"
[1] "N rep 36"
[1] 297  16


estimating size factors

estimating dispersions

gene-wise dispersion estimates: 4 workers

mean-dispersion relationship

final dispersion estimates, fitting model and testing: 4 workers

-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "stream"
[1] "N rep 32"
[1] 264  16


estimating size factors

estimating dispersions

gene-wise dispersion estimates: 4 workers

mean-dispersion relationship

final dispersion estimates, fitting model and testing: 4 workers

-- replacing outliers and refitting for 5 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "lag"
[1] "N rep 32"
[1] 263  16


estimating size factors

estimating dispersions

gene-wise dispersion estimates: 4 workers

mean-dispersion relationship

final dispersion estimates, fitting model and testing: 4 workers

